In [10]:
import numpy as np
import torch
import torchvision.models as models

from sklearn.metrics.pairwise import cosine_similarity

In [11]:
test_dataset = torch.load("data/test_dataset.pt")

In [7]:
import os

baseline = models.resnet50(pretrained=True)
    
n_features = baseline.fc.in_features
n_classes = test_dataset.get_n_classes()
    
if os.path.exists(f"models/baseline.pt"):
    baseline.load_state_dict(torch.load(f"models/baseline.pt"))

baseline.fc = torch.nn.Identity()

c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sebas\miniconda3\envs\trust-me\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([910, 2048]) from checkpoint, the shape in current model is torch.Size([1000, 2048]).
	size mismatch for fc.bias: copying a param with shape torch.Size([910]) from checkpoint, the shape in current model is torch.Size([1000]).

In [1]:
def get_accuracy(embeddings, targets):
    embeddings = np.array(embeddings)
    targets = np.array(targets)
    
    # Get cosine similarity for all embeddings
    similarity_matrix = cosine_similarity(embeddings)
    
    # Fill diagonal with inf
    np.fill_diagonal(similarity_matrix, -np.inf)
    
    # Find the index of max sim for each embedding
    most_similar_indices = np.argmax(similarity_matrix, axis=1)
    
    # Extract the labels of the most similar items
    predicted_labels = [targets[i] for i in most_similar_indices]
    n_correct = sum(1 for true, pred in zip(targets, predicted_labels) if true == pred)
    
    accuracy = 100 * (n_correct / len(embeddings))
    
    return accuracy, predicted_labels